# Simulation Basics

In this notebook, we'll introduce you to the basics of running a simulation of a manipulator arm in Colab. Specifically, we'll use a simulation of the [Kinova Gen3](https://www.kinovarobotics.com/en/products/gen3-robot) that we have in the lab. We've tried to make the simulation interface as close as possible to the hardware interface, so it's relatively easy to apply tools you've developed in simulation to the real robot. 

# Notebook Setup

Our simulation depends on a bunch of software (most notably [Drake](https://drake.mit.edu/)) that isn't installed by default on Google's servers. The following cell installs this software, which is pretty complex, so it will probably take a minute or two. You'll need to run a similar cell every time you use the simulator. Fortunately, once a cloud machine has been provisioned and the software intalled, it should remain availible to you for 12 hours or so. 

# Viewing the Simulator

# Controlling the Robot Manually

(try to pick up the block)